# Cleaning all 4 (excel) datasets in 4 steps

In [74]:
import pandas as pd
import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## 1. Cleaning the Sales dataset

In [75]:
"""
The problem arose that we wanted to enter the csv file into pgAdmin as a database. 
While importing the csv file, multiple errors arose that were not clear to understand. 
This notebook serves as a document to understand the data types in the columns and to clean the data.

"""

'\nThe problem arose that we wanted to enter the csv file into pgAdmin as a database. \nWhile importing the csv file, multiple errors arose that were not clear to understand. \nThis notebook serves as a document to understand the data types in the columns and to clean the data.\n\n'

In [76]:
# Import the excel file and look into the info
# Use the 'r' command to convert the string to a raw string, otherwise FileNotFoud errors may occur.

movie_sales = pd.read_excel(r"data\sales_dc.xlsx",
                            usecols=['year', 'release_date', 'title', 'worldwide_box_office', 'production_budget', 'url'])
print(movie_sales.info())
# Narrow the search down to only show the datatypes

movie_sales.dtypes
# Trying to get a look at the date frame

movie_sales.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30612 entries, 0 to 30611
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  30612 non-null  int64  
 1   release_date          30612 non-null  object 
 2   title                 30604 non-null  object 
 3   worldwide_box_office  21575 non-null  float64
 4   production_budget     4480 non-null   float64
 5   url                   30612 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 1.4+ MB
None


,year,release_date,title,worldwide_box_office,production_budget,url
0,2000,January 1st,Bakha Satang,76576.0,NaN,https://www.the-numbers.com/movie/Bakha-Satang...
1,2001,January 12th,Antitrust,17865209.0,30000000.0,https://www.the-numbers.com/movie/Antitrust
2,2000,January 28th,Santitos,NaN,NaN,https://www.the-numbers.com/movie/Santitos
3,2002,2002 (Wide) by,Frank McKlusky C.I.,NaN,NaN,https://www.the-numbers.com/movie/Frank-McKlus...
4,2002,January 25th,A Walk to Remember,46060861.0,11000000.0,https://www.the-numbers.com/movie/Walk-to-Reme...


In [77]:
movie_sales['year'] = movie_sales['year'].fillna(0).astype('int')

```
import pandas as pd

data = {'Year': [2023, 2022, 2021],
        'MonthDay': ['January 31st', 'February 15th', 'March 10th']}

df = pd.DataFrame(data)

# Extract the day and month parts from the 'MonthDay' column
df['Month'] = df['MonthDay'].str.extract(r'(\w+) \d{1,2}')
df['Day'] = df['MonthDay'].str.extract(r'(\d{1,2})')

# Combine 'Year', 'Month', and 'Day' columns into a new 'Date' column
df['Date'] = pd.to_datetime(df['Year'].astype(str) + ' ' + df['Month'] + ' ' + df['Day'], format='%Y %B %d')

# Drop the intermediate columns if desired
df.drop(['Year', 'MonthDay', 'Month', 'Day'], axis=1, inplace=True)

print(df)
```

In [78]:
movie_sales['month'] = movie_sales['release_date'].str.extract(r'(\w+) \d{1,2}')
movie_sales['day'] = movie_sales['release_date'].str.extract(r'(\d{1,2})')

movie_sales[['year', 'month', 'day']]


,year,month,day
0,2000,January,1
1,2001,January,12
2,2000,January,28
3,2002,NaN,20
4,2002,January,25
...,...,...,...
30607,2021,January,1
30608,2021,March,5
30609,2021,January,1
30610,2021,May,21


In [79]:
movie_sales['release_date'] = \
pd.to_datetime(
    (movie_sales['year'].astype('str') + ' ' + movie_sales['month'] + ' ' + movie_sales['day']),
    format='%Y %B %d',
    errors='coerce')

movie_sales['release_date'].sample(5)

6087    2017-06-30
28537   2020-02-14
21531   2006-03-17
3146    2016-10-07
27645   2019-04-05
Name: release_date, dtype: datetime64[ns]

In [80]:
# Add the column movie_id based on a URL string split
movie_sales['movie_id'] = movie_sales['url'].str.split('/').str.get(-1)

# Lower case the movie_id string
movie_sales['movie_id'] = movie_sales['movie_id'].str.lower()

# Move the column to the first position (index = 0)
firstcol = movie_sales.columns.to_list()
firstcol = ['movie_id'] + firstcol[:-1]
movie_sales = movie_sales[firstcol]

movie_sales = movie_sales[['movie_id', 'year', 'release_date', 'title', 'worldwide_box_office', 'production_budget', 'url']]

# Print updated dataframe head as an example
movie_sales.head()


,movie_id,year,release_date,title,worldwide_box_office,production_budget,url
0,bakha-satang-(s-korea),2000,2000-01-01,Bakha Satang,76576.0,NaN,https://www.the-numbers.com/movie/Bakha-Satang...
1,antitrust,2001,2001-01-12,Antitrust,17865209.0,30000000.0,https://www.the-numbers.com/movie/Antitrust
2,santitos,2000,2000-01-28,Santitos,NaN,NaN,https://www.the-numbers.com/movie/Santitos
3,frank-mcklusky-c-i,2002,NaT,Frank McKlusky C.I.,NaN,NaN,https://www.the-numbers.com/movie/Frank-McKlus...
4,walk-to-remember-a,2002,2002-01-25,A Walk to Remember,46060861.0,11000000.0,https://www.the-numbers.com/movie/Walk-to-Reme...


In [81]:
movie_sales.tail(20)

,movie_id,year,release_date,title,worldwide_box_office,production_budget,url
30592,life-on-denniston-plateau-(2012-new-zealand),2021,2021-01-14,Life On Denniston Plateau,1480.0,NaN,https://www.the-numbers.com/movie/Life-On-Denn...
30593,redemption-day-(morocco),2021,2021-01-08,Redemption Day,321.0,NaN,https://www.the-numbers.com/movie/Redemption-D...
30594,retrospective-exhibition-of-french-new-wave-fi...,2021,2021-01-08,A retrospective exhibition of French new wave ...,90000.0,NaN,https://www.the-numbers.com/movie/retrospectiv...
30595,reason-i-jump-the-(uk),2021,2021-01-08,The Reason I Jump,103653.0,NaN,https://www.the-numbers.com/movie/Reason-I-Jum...
30596,mil-y-una-las-(argentina),2021,2021-01-08,Las Mil y Una,3464.0,NaN,https://www.the-numbers.com/movie/Mil-y-Una-La...
30597,ji-hun-(2021-china),2021,2021-01-08,Ji Hun,17100000.0,NaN,https://www.the-numbers.com/movie/Ji-Hun-(2021...
30598,some-kind-of-heaven,2021,2021-01-08,Some Kind of Heaven,54864.0,NaN,https://www.the-numbers.com/movie/Some-Kind-of...
30599,gekijoban-bishojo-senshi-sera-mun-etanaru-part...,2021,2021-01-08,GekijÃ…ï¿½ban BishÃ…ï¿½jo Senshi SÃ„â€œrÃ„ï¿½ ...,NaN,NaN,https://www.the-numbers.com/movie/Gekijoban-Bi...
30600,i-blame-society-(2021),2021,2021-01-08,I Blame Society,12547.0,NaN,https://www.the-numbers.com/movie/I-Blame-Soci...
30601,average-joe-(2021),2021,2021-01-08,Average Joe,NaN,NaN,https://www.the-numbers.com/movie/Average-Joe-...


## 2. Cleaning the Metaclean dataset

In [82]:
# Import the excel file and look into the info
# Use the 'r' command to convert the string to a raw string, otherwise FileNotFoud errors may occur.

meta_clean = pd.read_excel(r"data/metaClean43Brightspace.xlsx")
print(meta_clean.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11364 entries, 0 to 11363
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   url        11364 non-null  object        
 1   title      11364 non-null  object        
 2   studio     11014 non-null  object        
 3   rating     10297 non-null  object        
 4   runtime    11109 non-null  float64       
 5   cast       7662 non-null   object        
 6   director   11350 non-null  object        
 7   genre      11344 non-null  object        
 8   summary    5467 non-null   object        
 9   awards     4387 non-null   object        
 10  metascore  11364 non-null  int64         
 11  userscore  9259 non-null   float64       
 12  RelDate    11364 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(9)
memory usage: 1.1+ MB
None


In [83]:
meta_clean.head()

,url,title,studio,rating,runtime,cast,director,genre,summary,awards,metascore,userscore,RelDate
0,https://www.metacritic.com/movie/!women-art-re...,!Women Art Revolution,Hotwire Productions,| Not Rated,83.0,NaN,Lynn Hershman-Leeson,Documentary,NaN,NaN,70,NaN,2011-06-01
1,https://www.metacritic.com/movie/10-cloverfiel...,10 Cloverfield Lane,Paramount Pictures,| PG-13,104.0,"John Gallagher Jr.,John Goodman,Mary Elizabeth...",Dan Trachtenberg,"Action,Sci-Fi,Drama,Mystery,Thriller,Horror","Waking up from a car accident, a young woman (...","#18MostDiscussedMovieof2016 , #1MostSharedMovi...",76,7.7,2016-03-11
2,https://www.metacritic.com/movie/10-items-or-less,10 Items or Less,Click Star,| R,82.0,"Jonah Hill,Morgan Freeman,Paz Vega",Brad Silberling,"Drama,Comedy,Romance",While researching a role as a supermarket mana...,NaN,54,5.8,2006-12-01
3,https://www.metacritic.com/movie/10-years,10 Years,Anchor Bay Entertainment,| R,100.0,"Channing Tatum,Chris Pratt,Jenna Dewan",Jamie Linden,"Drama,Comedy,Romance",NaN,NaN,61,6.9,2012-09-14
4,https://www.metacritic.com/movie/100-bloody-acres,100 Bloody Acres,Music Box Films,| Not Rated,91.0,NaN,Cameron Cairnes,"Horror,Comedy",Reg and Lindsay run an organic fertilizer busi...,NaN,63,7.5,2013-06-28


In [84]:
# Drop columns that we don't need for analysis
meta_clean = meta_clean.drop(columns=['studio', 'runtime', 'cast', 'director', 'summary', 'awards'])
# Look into the data types. This is important for the SQL database input

print(meta_clean.dtypes)

url                  object
title                object
rating               object
genre                object
metascore             int64
userscore           float64
RelDate      datetime64[ns]
dtype: object


In [85]:
# We create a movie id based on the url because this url has no special characters which will lead into more joins when joining the data later with the sales dataset.
#Split the URL to obtain the movie_id
meta_clean['movie_id'] = meta_clean['url'].str.split('/').str.get(-1)

# Lowercase the movie_id
meta_clean['movie_id'] = meta_clean['movie_id'].str.lower()

# Move new column to the first position in the table
firstcol = meta_clean.columns.to_list()
firstcol = ['movie_id'] + firstcol[:-1]
meta_clean = meta_clean[firstcol]


# Print updated head to ensure that it worked

meta_clean.head(6)

,movie_id,url,title,rating,genre,metascore,userscore,RelDate
0,!women-art-revolution,https://www.metacritic.com/movie/!women-art-re...,!Women Art Revolution,| Not Rated,Documentary,70,NaN,2011-06-01
1,10-cloverfield-lane,https://www.metacritic.com/movie/10-cloverfiel...,10 Cloverfield Lane,| PG-13,"Action,Sci-Fi,Drama,Mystery,Thriller,Horror",76,7.7,2016-03-11
2,10-items-or-less,https://www.metacritic.com/movie/10-items-or-less,10 Items or Less,| R,"Drama,Comedy,Romance",54,5.8,2006-12-01
3,10-years,https://www.metacritic.com/movie/10-years,10 Years,| R,"Drama,Comedy,Romance",61,6.9,2012-09-14
4,100-bloody-acres,https://www.metacritic.com/movie/100-bloody-acres,100 Bloody Acres,| Not Rated,"Horror,Comedy",63,7.5,2013-06-28
5,100-streets,https://www.metacritic.com/movie/100-streets,100 Streets,NaN,Drama,44,6.1,2017-01-13


In [86]:
meta_clean['genre'].unique()
meta_clean['genre'] = meta_clean['genre'].fillna('Genre unknown')
meta_clean['genre'].value_counts()

meta_clean['genre'].isnull().sum()

0

## 2.1 Age rating table

Carola

In [87]:
# Convert the data to CSV and save it at the specified location
# Again, be aware of the raw string 'r' to prevent errors

#Cleaning column rating CAROLA 
meta_clean['rating'] = meta_clean['rating'].str.replace('|', '')
meta_clean['rating'] = meta_clean['rating'].str.strip()
meta_clean.loc[meta_clean['rating'] == 'PG--13', 'rating'] = 'PG-13'
meta_clean.loc[meta_clean['rating'] == 'PG-13`', 'rating'] = 'PG-13'
meta_clean.loc[meta_clean['rating'] == 'Not Rated', 'rating'] = 'NR'
meta_clean.loc[meta_clean[
                   'rating'] == 'Unrated', 'rating'] = 'NR'  #Not rated and Unrated set equal to NR (also called UR) as they are the same category 
meta_clean.loc[meta_clean['rating'] == 'NR', 'rating'] = np.nan  #set as null since NR movies dont have an age rating
#line 9- 24  defining the subcategories for each age rating : all ages, 14+, 17+
meta_clean.loc[meta_clean['rating'] == 'Open', 'rating'] = 'All ages'
meta_clean.loc[meta_clean['rating'] == 'G', 'rating'] = 'All ages'
meta_clean.loc[meta_clean['rating'] == 'TV-G', 'rating'] = 'All ages'

meta_clean.loc[meta_clean['rating'] == 'Approved', 'rating'] = '14+'
meta_clean.loc[meta_clean['rating'] == 'TV-PG', 'rating'] = '14+'
meta_clean.loc[meta_clean['rating'] == 'PG', 'rating'] = '14+'
meta_clean.loc[meta_clean['rating'] == 'PG-13', 'rating'] = '14+'
meta_clean.loc[meta_clean['rating'] == 'TV-14', 'rating'] = '14+'
meta_clean.loc[meta_clean['rating'] == 'M/PG', 'rating'] = '14+'

meta_clean.loc[meta_clean['rating'] == 'R', 'rating'] = '17+'
meta_clean.loc[meta_clean['rating'] == 'MA-17', 'rating'] = '17+'
meta_clean.loc[meta_clean['rating'] == 'TV-MA', 'rating'] = '17+'
meta_clean.loc[meta_clean['rating'] == 'NC-17', 'rating'] = '17+'
meta_clean.loc[meta_clean['rating'] == 'M', 'rating'] = '17+'

distinct_ratings = meta_clean['rating'].unique()
print([rating for rating in distinct_ratings])


[nan, '14+', '17+', 'All ages']


C:\Users\Martijn\AppData\Local\Temp\ipykernel_4680\1754717522.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  meta_clean['rating'] = meta_clean['rating'].str.replace('|', '')


In [88]:
# creating Rating_id (from age rating)  Carola 
distinct_ratings = meta_clean['rating'].unique()  # Get the unique age ratings from the rating column
age_ratings = list(distinct_ratings)  #putting them in a list 
rating_id_mapping = {genre: idx for idx, genre in enumerate(distinct_ratings)}  #mapping ratings to unique ratings IDs  
meta_clean['rating_id'] = meta_clean['rating'].map(rating_id_mapping)  #adding column rating_id to dataset
distinct_ratings_id = meta_clean['rating_id'].unique()  #get the unique age ratings id from the new rating_id column
age_ratings_id = list(distinct_ratings_id)  #creating a python list with distinct genre _ids
ratings_ratings_id_dict = {key: value for key, value in
                           zip(distinct_ratings, age_ratings)}  # Zip the two lists into a dictionary
meta_clean.replace(ratings_ratings_id_dict, regex=True)
meta_clean.head()

,movie_id,url,title,rating,genre,metascore,userscore,RelDate,rating_id
0,!women-art-revolution,https://www.metacritic.com/movie/!women-art-re...,!Women Art Revolution,NaN,Documentary,70,NaN,2011-06-01,0
1,10-cloverfield-lane,https://www.metacritic.com/movie/10-cloverfiel...,10 Cloverfield Lane,14+,"Action,Sci-Fi,Drama,Mystery,Thriller,Horror",76,7.7,2016-03-11,1
2,10-items-or-less,https://www.metacritic.com/movie/10-items-or-less,10 Items or Less,17+,"Drama,Comedy,Romance",54,5.8,2006-12-01,2
3,10-years,https://www.metacritic.com/movie/10-years,10 Years,17+,"Drama,Comedy,Romance",61,6.9,2012-09-14,2
4,100-bloody-acres,https://www.metacritic.com/movie/100-bloody-acres,100 Bloody Acres,NaN,"Horror,Comedy",63,7.5,2013-06-28,0


In [89]:
age_rating_table = pd.DataFrame(data={'rating':distinct_ratings}).reset_index().rename(columns={'index':'rating_id'})
age_rating_table

,rating_id,rating
0,0,NaN
1,1,14+
2,2,17+
3,3,All ages


In [90]:
age_rating_table.to_csv('Cleaned data/age_rating_table.csv', sep=',', index=False)

In [91]:
meta_clean.drop(columns=['rating'],
                inplace=True)  #drop rating column and making changes permanent by using inplace = True 
meta_clean.sample(10)

,movie_id,url,title,genre,metascore,userscore,RelDate,rating_id
7163,samson-and-delilah,https://www.metacritic.com/movie/samson-and-de...,Samson and Delilah,"Drama,Romance",75,7.7,2010-10-15,0
8826,the-gallows,https://www.metacritic.com/movie/the-gallows,The Gallows,"Thriller,Horror",30,6.1,2015-07-10,2
10263,third-world-cop,https://www.metacritic.com/movie/third-world-cop,Third World Cop,"Action,Drama,Thriller",28,NaN,2000-04-14,2
4697,killers,https://www.metacritic.com/movie/killers,Killers,"Action,Thriller,Comedy,Romance",21,3.9,2010-06-04,1
10858,war-for-the-planet-of-the-apes,https://www.metacritic.com/movie/war-for-the-p...,War for the Planet of the Apes,"Action,Adventure,Sci-Fi,Drama,Thriller,War",82,8.0,2017-07-14,1
2594,downfall,https://www.metacritic.com/movie/downfall,Downfall,"Biography,Drama,History,War",82,8.6,2005-02-18,2
5807,music,https://www.metacritic.com/movie/music,Music,"Drama,Musical",23,6.6,2021-02-12,1
7659,southside-with-you,https://www.metacritic.com/movie/southside-wit...,Southside with You,"Biography,Drama,Romance",74,5.9,2016-08-26,1
2443,dick-johnson-is-dead,https://www.metacritic.com/movie/dick-johnson-...,Dick Johnson Is Dead,"Drama,Documentary",89,7.5,2020-10-02,1
10262,third-person,https://www.metacritic.com/movie/third-person,Third Person,"Drama,Romance",38,5.0,2014-06-20,2


## 2.2 Genre table

### Unique list of genres

Carola

In [92]:
meta_clean.head()

,movie_id,url,title,genre,metascore,userscore,RelDate,rating_id
0,!women-art-revolution,https://www.metacritic.com/movie/!women-art-re...,!Women Art Revolution,Documentary,70,NaN,2011-06-01,0
1,10-cloverfield-lane,https://www.metacritic.com/movie/10-cloverfiel...,10 Cloverfield Lane,"Action,Sci-Fi,Drama,Mystery,Thriller,Horror",76,7.7,2016-03-11,1
2,10-items-or-less,https://www.metacritic.com/movie/10-items-or-less,10 Items or Less,"Drama,Comedy,Romance",54,5.8,2006-12-01,2
3,10-years,https://www.metacritic.com/movie/10-years,10 Years,"Drama,Comedy,Romance",61,6.9,2012-09-14,2
4,100-bloody-acres,https://www.metacritic.com/movie/100-bloody-acres,100 Bloody Acres,"Horror,Comedy",63,7.5,2013-06-28,0


In [93]:
#Cleaning genre column 
#extracting unique movie genres as in the dataset multiple genres were on the same row and considered as a genre on its own. If left unchanged there would 1312 different genres which would impact the validity of the analysis. 
unique_genres = []
for genres_contaminated in meta_clean['genre']:
    if not isinstance(genres_contaminated, str):
        continue  #skipping to next element if genre is not a string, as the float values (nan) belong to movies not being assigned a genre 
    if ',' in genres_contaminated:  #if a movie has multiple genres (comma separated)
        genres = genres_contaminated.split(',')  #splitting genres on comma
        for genre in genres:
            if genre not in unique_genres:
                unique_genres.append(genre)  # if separated genre not yet in the list append it
    else:  #if a movie has only one genre
        if genres_contaminated not in unique_genres:
            unique_genres.append(genres_contaminated)  #if genre is not in list append it
print(unique_genres)



['Documentary', 'Action', 'Sci-Fi', 'Drama', 'Mystery', 'Thriller', 'Horror', 'Comedy', 'Romance', 'Adventure', 'Fantasy', 'Family', 'Crime', 'War', 'History', 'Sport', 'Biography', 'Music', 'News', 'Musical', 'Western', 'Animation', 'Talk-Show', 'Adult', 'Short', 'Genre unknown', 'Reality-TV', 'Film-Noir']


Mees

In [94]:
genre_table = pd.DataFrame(data={'genre': unique_genres})
genre_table.reset_index(inplace=True)

genre_table

,index,genre
0,0,Documentary
1,1,Action
2,2,Sci-Fi
3,3,Drama
4,4,Mystery
5,5,Thriller
6,6,Horror
7,7,Comedy
8,8,Romance
9,9,Adventure


In [95]:
genre_table['genre_id'] = genre_table['index'] + 1
genre_table.drop('index', axis=1, inplace=True)
genre_table[['genre_id', 'genre']].head()

,genre_id,genre
0,1,Documentary
1,2,Action
2,3,Sci-Fi
3,4,Drama
4,5,Mystery


In [96]:
genre_table.to_csv('Cleaned data/genre_table.csv', sep=',', index=False)
print('Table has been successfully exported to csv into cleaned data folder')

Table has been successfully exported to csv into cleaned data folder


### Creating the movie_genre_junction_table

In [97]:
# Creating the movie genre junction table
movie_genre_junction_table = meta_clean[['movie_id', 'genre']].copy()
movie_genre_junction_table.head()

,movie_id,genre
0,!women-art-revolution,Documentary
1,10-cloverfield-lane,"Action,Sci-Fi,Drama,Mystery,Thriller,Horror"
2,10-items-or-less,"Drama,Comedy,Romance"
3,10-years,"Drama,Comedy,Romance"
4,100-bloody-acres,"Horror,Comedy"


In [98]:
# Convert the genres that are stored in the genre column to a list in this column so that we can explode it into multiple rows.
movie_genre_junction_table['genre'] = movie_genre_junction_table['genre'].str.split(',').tolist()
movie_genre_junction_table

,movie_id,genre
0,!women-art-revolution,[Documentary]
1,10-cloverfield-lane,"[Action, Sci-Fi, Drama, Mystery, Thriller, Hor..."
2,10-items-or-less,"[Drama, Comedy, Romance]"
3,10-years,"[Drama, Comedy, Romance]"
4,100-bloody-acres,"[Horror, Comedy]"
...,...,...
11359,zoolander-2,[Comedy]
11360,zoom,"[Action, Adventure, Sci-Fi, Family]"
11361,zoom-2016,"[Drama, Comedy, Animation]"
11362,zootopia,"[Action, Adventure, Comedy, Crime, Animation, ..."


In [99]:
# The epxlode method splits the lists stored in the genre table and creates a row for each genre in the cell.
movie_genre_junction_table = movie_genre_junction_table.explode('genre')
movie_genre_junction_table

,movie_id,genre
0,!women-art-revolution,Documentary
1,10-cloverfield-lane,Action
1,10-cloverfield-lane,Sci-Fi
1,10-cloverfield-lane,Drama
1,10-cloverfield-lane,Mystery
...,...,...
11362,zootopia,Animation
11362,zootopia,Family
11363,zus-zo,Fantasy
11363,zus-zo,Comedy


In [100]:
keys = genre_table['genre']
values = genre_table['genre_id']

replacement_dictionary = {key: value for key, value in zip(keys, values)}

movie_genre_junction_table['genre_id'] = movie_genre_junction_table['genre'].replace(replacement_dictionary, regex=True).astype('int')
movie_genre_junction_table

,movie_id,genre,genre_id
0,!women-art-revolution,Documentary,1
1,10-cloverfield-lane,Action,2
1,10-cloverfield-lane,Sci-Fi,3
1,10-cloverfield-lane,Drama,4
1,10-cloverfield-lane,Mystery,5
...,...,...,...
11362,zootopia,Animation,22
11362,zootopia,Family,12
11363,zus-zo,Fantasy,11
11363,zus-zo,Comedy,8


In [101]:
movie_genre_junction_table[['movie_id', 'genre_id']].to_csv('Cleaned data/movie_genre_junction_table.csv',
                                                            index=False,
                                                            sep=',')

=============================================================================================
## 3. Creating movies table by merging sales and metaclean

Left join is used because we only want the matching titles. All titles that have a different spelling will not be added because this is out of the scope of this assignment. (Otherwise we could have solved this with Fuzzy matching.)

In [102]:
#movie_sales.head()

In [103]:
#meta_clean.head()

In [104]:
# @Mees, can you please check if this works? xoxo Martijn and Carola :)
#movie_table = movie_sales.merge(meta_clean[['movie_id','metascore','userscore','rating_id']],
                               # how='left', left_on='movie_id', right_on='movie_id')

#movie_table.head()

In [105]:
# movie_table['release_date'] = pd.to_datetime(movie_table['RelDate'])

"""

movie_table = movie_table[['movie_id',
             'title', 
             'release_date',
             'metascore',
             'userscore',
             'production_budget',
             'worldwide_box_office',
            'rating_id']]
 
movie_table.to_csv('Cleaned data/movie_table.csv', sep=',', index=False)
movie_table.head()

"""

=============================================================================================

## 4. Cleaning the review datasets (expert and user)# 

## 4.1 Reading datasets

Mees

In [106]:
# # (Mees)
# expert = pd.read_excel('data/ExpertReviewsClean43LIWC.xlsx',
#                         dtype={'url':'str',
#                               'idvscore':'str',
#                               'reviewer':'str',
#                               'dateP':'str',
#                               'posemo':'float',
#                               'negemo':'float'},
#                         na_filter=False
#                         )
# expert.head()

In [107]:
# # (Mees)
# user = pd.read_excel('data/UserReviewsClean43LIWC.xlsx',
#                        dtype={'url':'str',
#                               'idvscore':'str',
#                               'reviewer':'str',
#                               'dateP':'str',
#                               'posemo':'float',
#                               'negemo':'float'},
#                        na_filter=False
#                        )
# user.head()

In [108]:
# # I decided to combine the expert and user reviews to improve database efficiency (Mees)
# reviews = pd.concat([expert, user], keys=['expert', 'user']).reset_index()
# 
# reviews.rename(columns={'level_0':'review_id', 'level_1':'review_id'}, inplace=True)
# 
# reviews['review_id'] = reviews['review_id'] + 1
# 
# reviews.to_parquet('data/reviews.parquet.gzip',
#                    compression='gzip',
#                    index=False)
# 
# reviews.tail()

In [109]:
# (Mees)
reviews = pd.read_parquet('data/reviews.parquet.gzip')

reviews.sample(10)

,Reviewer_role,review_id,url,idvscore,reviewer,dateP,Rev,WC,posemo,negemo
508637,user,269665,https://www.metacritic.com/movie/the-texas-cha...,7,'imthenoob',"'Feb 20, 2012'",'Suprisingly better then the previous remake ...,NaN,13.95,3.49
399212,user,160240,https://www.metacritic.com/movie/bpm-beats-per...,8,'MichaelObe',"'Oct 20, 2017'","'In the early 1990s, the AIDS epidemic in Fra...",NaN,3.34,3.08
342997,user,104025,https://www.metacritic.com/movie/joker,1,'LucasMolto',"'Oct 22, 2019'",'Awful movie. Just misery porn 4 upper class ...,NaN,5.56,11.11
351163,user,112191,https://www.metacritic.com/movie/money-monster,6,'LeZee',"'Sep 9, 2016'",'What if the experts are wrong! And how it af...,NaN,6.44,1.38
4098,expert,4099,https://www.metacritic.com/movie/the-battle-of...,20,'A.O. Scott',None,'May lead to a new axiom: success has many fa...,15.0,6.67,6.67
211891,expert,211892,https://www.metacritic.com/movie/bukowski-born...,70,'Kevin Thomas',None,'Accomplishes beautifully what it sets out to...,27.0,0.00,0.00
280840,user,41868,https://www.metacritic.com/movie/need-for-speed,5,'yashdan',"'Aug 4, 2014'",'An OK movie with an OK story with very poor ...,NaN,15.38,7.69
433164,user,194192,https://www.metacritic.com/movie/star-trek-int...,6,'hapycampr2002',"'May 27, 2013'","'This review contains spoilers, click expand ...",NaN,2.17,3.48
105303,expert,105304,https://www.metacritic.com/movie/the-emoji-movie,0,'Alonso Duralde',"'Jul 27, 2017'",'It is a soul-crushing disaster because it la...,51.0,1.96,1.96
367070,user,128098,https://www.metacritic.com/movie/avengers-endgame,2,'TrevorsView',"'May 2, 2019'","'This review contains spoilers, click expand ...",NaN,4.06,3.82


In [110]:
reviews

,Reviewer_role,review_id,url,idvscore,reviewer,dateP,Rev,WC,posemo,negemo
0,expert,1,https://www.metacritic.com/movie/bronson,100,"""Andrew O'Hehir""",None,'Bronson owes a little or a lot to Kubrick s ...,25.0,0.00,0.00
1,expert,2,https://www.metacritic.com/movie/bronson,90,'A.O. Scott',None,'Bronson invites you to admire its protagonis...,30.0,6.67,0.00
2,expert,3,https://www.metacritic.com/movie/bronson,90,None,None,'Whether it s Peterson/Bronson s more theatri...,40.0,0.00,2.50
3,expert,4,https://www.metacritic.com/movie/bronson,83,'Noel Murray',None,'There are two Bronsons on display here: the ...,39.0,5.13,5.13
4,expert,5,https://www.metacritic.com/movie/bronson,80,'Joshua Rothkopf',None,'Refn has somehow found his way to an authent...,24.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
558630,user,319658,https://www.metacritic.com/movie/spirited-away,10,'Zenflar',"'Sep 6, 2021'",'A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+...,NaN,0.00,0.00
558631,user,319659,https://www.metacritic.com/movie/spirited-away,10,'juiliopaublito',"'Nov 25, 2021'",'in my opinion spirited away is one of the be...,NaN,4.55,0.00
558632,user,319660,https://www.metacritic.com/movie/spirited-away,10,'PorridgeBoy3000',"'Dec 31, 2021'",'Usually I don t like watching movies that mu...,NaN,5.71,0.00
558633,user,319661,https://www.metacritic.com/movie/spirited-away,10,'jamesfhall',"'Jan 20, 2022'",'Studio Ghibli s gripping masterpiece Spirite...,NaN,8.33,0.00


## 4.2 Cleaning

Martijn

In [111]:
# Create a unique review ID for every review by using the unique() function
# Make changes permanent by using 'inplace=True'
reviews['Rev'].unique()
reviews

,Reviewer_role,review_id,url,idvscore,reviewer,dateP,Rev,WC,posemo,negemo
0,expert,1,https://www.metacritic.com/movie/bronson,100,"""Andrew O'Hehir""",None,'Bronson owes a little or a lot to Kubrick s ...,25.0,0.00,0.00
1,expert,2,https://www.metacritic.com/movie/bronson,90,'A.O. Scott',None,'Bronson invites you to admire its protagonis...,30.0,6.67,0.00
2,expert,3,https://www.metacritic.com/movie/bronson,90,None,None,'Whether it s Peterson/Bronson s more theatri...,40.0,0.00,2.50
3,expert,4,https://www.metacritic.com/movie/bronson,83,'Noel Murray',None,'There are two Bronsons on display here: the ...,39.0,5.13,5.13
4,expert,5,https://www.metacritic.com/movie/bronson,80,'Joshua Rothkopf',None,'Refn has somehow found his way to an authent...,24.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
558630,user,319658,https://www.metacritic.com/movie/spirited-away,10,'Zenflar',"'Sep 6, 2021'",'A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+...,NaN,0.00,0.00
558631,user,319659,https://www.metacritic.com/movie/spirited-away,10,'juiliopaublito',"'Nov 25, 2021'",'in my opinion spirited away is one of the be...,NaN,4.55,0.00
558632,user,319660,https://www.metacritic.com/movie/spirited-away,10,'PorridgeBoy3000',"'Dec 31, 2021'",'Usually I don t like watching movies that mu...,NaN,5.71,0.00
558633,user,319661,https://www.metacritic.com/movie/spirited-away,10,'jamesfhall',"'Jan 20, 2022'",'Studio Ghibli s gripping masterpiece Spirite...,NaN,8.33,0.00


In [112]:
#Rename columns
reviews = reviews.rename(columns={'Reviewer_role': 'role_id'})
# Use datatype and head command to look at the results
reviews.head()

,role_id,review_id,url,idvscore,reviewer,dateP,Rev,WC,posemo,negemo
0,expert,1,https://www.metacritic.com/movie/bronson,100,"""Andrew O'Hehir""",None,'Bronson owes a little or a lot to Kubrick s ...,25.0,0.00,0.00
1,expert,2,https://www.metacritic.com/movie/bronson,90,'A.O. Scott',None,'Bronson invites you to admire its protagonis...,30.0,6.67,0.00
2,expert,3,https://www.metacritic.com/movie/bronson,90,None,None,'Whether it s Peterson/Bronson s more theatri...,40.0,0.00,2.50
3,expert,4,https://www.metacritic.com/movie/bronson,83,'Noel Murray',None,'There are two Bronsons on display here: the ...,39.0,5.13,5.13
4,expert,5,https://www.metacritic.com/movie/bronson,80,'Joshua Rothkopf',None,'Refn has somehow found his way to an authent...,24.0,0.00,0.00


In [113]:
reviews['role_id'] = reviews['role_id'].replace({'expert':1, 'user':0}, regex=True)
reviews.head()

,role_id,review_id,url,idvscore,reviewer,dateP,Rev,WC,posemo,negemo
0,1,1,https://www.metacritic.com/movie/bronson,100,"""Andrew O'Hehir""",None,'Bronson owes a little or a lot to Kubrick s ...,25.0,0.00,0.00
1,1,2,https://www.metacritic.com/movie/bronson,90,'A.O. Scott',None,'Bronson invites you to admire its protagonis...,30.0,6.67,0.00
2,1,3,https://www.metacritic.com/movie/bronson,90,None,None,'Whether it s Peterson/Bronson s more theatri...,40.0,0.00,2.50
3,1,4,https://www.metacritic.com/movie/bronson,83,'Noel Murray',None,'There are two Bronsons on display here: the ...,39.0,5.13,5.13
4,1,5,https://www.metacritic.com/movie/bronson,80,'Joshua Rothkopf',None,'Refn has somehow found his way to an authent...,24.0,0.00,0.00


In [114]:
# Operation successful
reviews.head()

,role_id,review_id,url,idvscore,reviewer,dateP,Rev,WC,posemo,negemo
0,1,1,https://www.metacritic.com/movie/bronson,100,"""Andrew O'Hehir""",None,'Bronson owes a little or a lot to Kubrick s ...,25.0,0.00,0.00
1,1,2,https://www.metacritic.com/movie/bronson,90,'A.O. Scott',None,'Bronson invites you to admire its protagonis...,30.0,6.67,0.00
2,1,3,https://www.metacritic.com/movie/bronson,90,None,None,'Whether it s Peterson/Bronson s more theatri...,40.0,0.00,2.50
3,1,4,https://www.metacritic.com/movie/bronson,83,'Noel Murray',None,'There are two Bronsons on display here: the ...,39.0,5.13,5.13
4,1,5,https://www.metacritic.com/movie/bronson,80,'Joshua Rothkopf',None,'Refn has somehow found his way to an authent...,24.0,0.00,0.00


In [115]:
# Drop columns that we don't need
# Make changes permanent by using 'inplace=True'
reviews.drop(columns=['idvscore', 'reviewer', 'dateP', 'Rev', 'WC'], inplace=True)
reviews.head()

,role_id,review_id,url,posemo,negemo
0,1,1,https://www.metacritic.com/movie/bronson,0.00,0.00
1,1,2,https://www.metacritic.com/movie/bronson,6.67,0.00
2,1,3,https://www.metacritic.com/movie/bronson,0.00,2.50
3,1,4,https://www.metacritic.com/movie/bronson,5.13,5.13
4,1,5,https://www.metacritic.com/movie/bronson,0.00,0.00


In [116]:
# remove the url of https://... (Martijn)
reviews['url'] = reviews['url'].str.split('/').str.get(-1).replace('-', ' ', regex=True)

reviews.head()

,role_id,review_id,url,posemo,negemo
0,1,1,bronson,0.00,0.00
1,1,2,bronson,6.67,0.00
2,1,3,bronson,0.00,2.50
3,1,4,bronson,5.13,5.13
4,1,5,bronson,0.00,0.00


Mees

In [117]:
#example with year in url that must be deleted (Mees)
reviews.iloc[17238, :]

role_id                 1
review_id           17239
url          carnage 2011
posemo                0.0
negemo               4.17
Name: 17238, dtype: object

In [118]:
# delete years in url column so there wont be double years after i add them. (Mees)
reviews['url'] = reviews['url'].str.replace(r'(-\b\d{4}\b)', '', regex=True)
reviews.iloc[17238, :]

role_id                 1
review_id           17239
url          carnage 2011
posemo                0.0
negemo               4.17
Name: 17238, dtype: object

In [119]:
#check what else must be cleaned
reviews.head()

,role_id,review_id,url,posemo,negemo
0,1,1,bronson,0.00,0.00
1,1,2,bronson,6.67,0.00
2,1,3,bronson,0.00,2.50
3,1,4,bronson,5.13,5.13
4,1,5,bronson,0.00,0.00


In [120]:
# Clean the posemo and negomo to correct datatypes
reviews['posemo'] = pd.to_numeric(reviews['posemo'],
                                  downcast='float',
                                  errors='coerce')

reviews['negemo'] = pd.to_numeric(reviews['negemo'],
                                  downcast='float',
                                  errors='coerce')

In [121]:
# Create derived column Overall sentiment (Carola & Mees)
reviews.loc[(reviews['posemo'] - reviews['negemo']) < 0, 'Overall_sentiment'] = 'negative' # Mees & Carola
reviews.loc[(reviews['posemo'] - reviews['negemo']) == 0, 'Overall_sentiment'] = 'neutral' # Mees & Carola
reviews.loc[(reviews['posemo'] - reviews['negemo']) > 0, 'Overall_sentiment'] = 'positive' # Mees & Carola
reviews.head() # Mees & Carola

,role_id,review_id,url,posemo,negemo,Overall_sentiment
0,1,1,bronson,0.00,0.00,neutral
1,1,2,bronson,6.67,0.00,positive
2,1,3,bronson,0.00,2.50,negative
3,1,4,bronson,5.13,5.13,neutral
4,1,5,bronson,0.00,0.00,neutral


In [122]:
reviews.to_csv('Cleaned data/review_table.csv', sep=',', index=False)
reviews.tail()

,role_id,review_id,url,posemo,negemo,Overall_sentiment
558630,0,319658,spirited away,0.00,0.0,neutral
558631,0,319659,spirited away,4.55,0.0,positive
558632,0,319660,spirited away,5.71,0.0,positive
558633,0,319661,spirited away,8.33,0.0,positive
558634,0,319662,spirited away,7.14,0.0,positive


### Creating role table
Martijn

In [123]:
# Create simple dataframe for the role table
role_data = [('expert', 1), ('user', 0)]
role_table = pd.DataFrame(role_data, columns=['(expert, user)', '(0,1)'])
role_table.rename(columns={'(expert, user)':'reviewer_role','(0,1)':'role_id'},inplace=True)

role_table.to_csv('Cleaned data/role_table.csv', sep=',', index=False)
role_table.head()


,reviewer_role,role_id
0,expert,1
1,user,0


### Fuzzy Matching
Inspiration code: https://saturncloud.io/blog/is-it-possible-to-do-fuzzy-match-merge-with-python-pandas/

In [124]:
#Fuzzy match met movie_id van movies table (ToDo)
# 

In [125]:
# urls_df = pd.DataFrame(data=reviews['url'].unique(),
#              columns=['key'])
# urls_df

In [126]:
# movie_id_df = pd.DataFrame(data=movie_sales['movie_id'].unique(),
#              columns=['key'])
# 
# movie_id_df.replace('-', ' ', regex=True, inplace=True)
# movie_id_df

In [127]:
# movie_id_df_subset = movie_id_df[0:100]

In [128]:
# def fuzzy_match(str1, str2):
#     return fuzz.token_set_ratio(str1, str2)
# 
# 
# def match_dataframe(df1, df2, threshold=80):
#     matches = []
#     for i, row in df1.iterrows():
#         match = process.extractOne(row['key'], df2['key'], scorer=fuzzy_match)
#         if match[1] >= threshold:
#             matches.append([i, match[0], match[1]])
#     return matches
# 
# matches = match_dataframe(urls_df, movie_id_df_subset)
# matches

In [129]:
# merged_df = pd.DataFrame(matches, columns=['index1', 'index2', 'score'])
# merged_df

In [130]:
# merged_df = pd.merge(merged_df, urls_df, left_on='index1', right_index=True)
# merged_df

In [131]:
# merged_df = pd.merge(merged_df, movie_id_df_subset, left_on='index2', right_index=True)
# merged_df

In [132]:
# def print_max_length(data_frame, column_name):
#     max_length = data_frame[~data_frame[column_name].isnull()][column_name].apply(len).max()
#     print(f"Maximum character length of column {column_name} is {max_length} characters")
# 
# print_max_length(movie_table, 'movie_id')


## 3. Creating movies table by merging sales and metaclean

Left join is used because we only want the matching titles. All titles that have a different spelling will not be added 
because this is out of the scope of this assignment.(Otherwise we could have solved this with Fuzzy matching.)




In [133]:
movie_sales.head()
meta_clean.head()
movie_table = movie_sales.merge(meta_clean[['movie_id','metascore','userscore','rating_id']],
                               how='left', left_on='movie_id', right_on='movie_id')
movie_table.head()

,movie_id,year,release_date,title,worldwide_box_office,production_budget,url,metascore,userscore,rating_id
0,bakha-satang-(s-korea),2000,2000-01-01,Bakha Satang,76576.0,NaN,https://www.the-numbers.com/movie/Bakha-Satang...,NaN,NaN,NaN
1,antitrust,2001,2001-01-12,Antitrust,17865209.0,30000000.0,https://www.the-numbers.com/movie/Antitrust,31.0,8.7,1.0
2,santitos,2000,2000-01-28,Santitos,NaN,NaN,https://www.the-numbers.com/movie/Santitos,68.0,NaN,2.0
3,frank-mcklusky-c-i,2002,NaT,Frank McKlusky C.I.,NaN,NaN,https://www.the-numbers.com/movie/Frank-McKlus...,NaN,NaN,NaN
4,walk-to-remember-a,2002,2002-01-25,A Walk to Remember,46060861.0,11000000.0,https://www.the-numbers.com/movie/Walk-to-Reme...,NaN,NaN,NaN


In [134]:
# movie_table['release_date'] = pd.to_datetime(movie_table['RelDate'])
movie_table = movie_table[['movie_id',
                           'title',
                           'release_date',
                           'metascore',
                           'userscore',
                           'production_budget',
                           'worldwide_box_office',
                           'rating_id']]

movie_table.to_csv('Cleaned data/movie_table.csv', sep=',', index=False)
movie_table.head()

,movie_id,title,release_date,metascore,userscore,production_budget,worldwide_box_office,rating_id
0,bakha-satang-(s-korea),Bakha Satang,2000-01-01,NaN,NaN,NaN,76576.0,NaN
1,antitrust,Antitrust,2001-01-12,31.0,8.7,30000000.0,17865209.0,1.0
2,santitos,Santitos,2000-01-28,68.0,NaN,NaN,NaN,2.0
3,frank-mcklusky-c-i,Frank McKlusky C.I.,NaT,NaN,NaN,NaN,NaN,NaN
4,walk-to-remember-a,A Walk to Remember,2002-01-25,NaN,NaN,11000000.0,46060861.0,NaN
